In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train = pd.read_csv('train_final.csv')
test = pd.read_csv('test_final.csv')

Анализ данных

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
print(train.shape, test.shape)

(44638, 29) (23525, 28)


In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
train['is_canceled'].value_counts()

1.   В данных есть разные типы данных, нужно устранить категориальные признаки
2.   Отсутствуют пропуски
3.   Нужно восстановить колонку is_canceled
4.   Задача решается с помощью логистической регрессии
5.   Данные несбалансированные






Предобработка данных

In [ ]:
train = train.drop(columns=['reservation_status_date', 'country', 'arrival_date_year', 'reservation_status_date'])
test = test.drop(columns=['reservation_status_date', 'country', 'arrival_date_year', 'reservation_status_date'])

In [ ]:
categ = train.loc[:,train.dtypes == 'object']

In [ ]:
for i in categ:
    label_encoder = LabelEncoder()
    label_encoder.fit(train[i])
    train[i] = label_encoder.transform(train[i])

In [ ]:
for i in categ:
    label_encoder = LabelEncoder()
    label_encoder.fit(test[i])
    test[i] = label_encoder.transform(test[i])

In [ ]:
test.loc[:,test.dtypes == 'object']

Создание модели

In [ ]:
X = train.drop(columns = ['is_canceled'])
y = train['is_canceled']

In [ ]:
X_nm = pd.DataFrame(MinMaxScaler().fit_transform(X), columns = X.columns, index = X.index)

In [ ]:
X_nm.head()

,hotel,lead_time,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,...,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
0,1.0,0.004071,0.272727,0.115385,0.366667,0.0000,0.050,0.036364,0.000000,0.0,...,0.0,0.500,0.5,0.0,0.0,0.0,0.666667,0.158824,0.125,0.0
1,0.0,0.067843,0.818182,0.884615,0.566667,0.0625,0.050,0.036364,0.000000,0.0,...,0.0,0.000,0.0,0.0,0.0,0.0,0.666667,0.183529,0.000,0.4
2,0.0,0.018996,1.000000,0.692308,0.200000,0.0625,0.125,0.036364,0.666667,0.0,...,0.0,0.625,0.5,0.0,0.0,0.0,0.000000,0.325490,0.000,0.6
3,0.0,0.024423,0.181818,0.942308,0.166667,0.1250,0.000,0.036364,0.000000,0.0,...,0.0,0.000,0.0,0.0,0.0,0.0,0.666667,0.209804,0.000,0.0
4,0.0,0.356852,1.000000,0.750000,0.933333,0.1250,0.100,0.036364,0.000000,0.0,...,0.0,0.000,0.0,0.0,0.0,0.0,0.666667,0.196667,0.000,0.0


In [ ]:
model = LogisticRegression()
model.fit(X_nm, y)
pred = model.predict(X)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
print(accuracy_score(pred, y))
print(precision_score(pred, y))
print(recall_score(pred, y))

0.3036650387562167
0.9993224932249323
0.29933031184468645


In [ ]:
plot_confusion_matrix(model, x, y)
plt.show()

In [ ]:
test_pred = model.predict(test)
test_pred = pd.DataFrame(test_pred, columns = ['is_canceled'])
test_pred = test_pred.reset_index()
test_pred.to_csv('solution.csv', index = False)

Какие шаги можно сделать, чтобы улучшить результат до 0,6+

1. решить, какие данные удалить, какие оставить
2. кодировать все данные одним из двух кодировшиков
3. масштабировать данные
4. сбалансировать данные
